In [1]:
import requests
import KOBO_URL
import time
from bs4 import BeautifulSoup

# スクレイピング対象のサイトのURLを取得
url1 = KOBO_URL.KOBO_URL1
res = requests.get(url1)

# WebサイトのHTML要素を取得
soup = BeautifulSoup(res.text, 'html.parser')

# li要素を取得
soup_li = soup.find_all('li', attrs={'class': 'contest-list-item'})

# 応募リンクを配列に格納する
link_list = []
while_cnt = 0
while True:
    try:
        soup_a = soup_li[while_cnt].find('a')
        soup_a = soup_a.get('href')
        link_list.append(soup_a)
        
    except IndexError as e:
        break
        
    while_cnt += 1


kobo_title_list = []
kobo_feature_list = []
kobo_info_list = []


for n in range(len(link_list)):
# for n in range(2):
    # 情報を取得する公募ページに移動してHTTPを取得
    kobo_url = link_list[n]
    res_kobo = requests.get(kobo_url)
    soup_kobo = BeautifulSoup(res_kobo.text, 'html.parser')
    soup_kobo_dl = soup_kobo.find_all('dl')
    
    while_cnt = 0
    while True:
        try:
            soup_kobo_dt = soup_kobo_dl[while_cnt].find_all('dt')
            if '締切' == soup_kobo_dt[0].text:
                break
        except IndexError as e:
            break
        while_cnt += 1
    
    soup_kobo_dd = soup_kobo_dl[while_cnt].find_all('dd')
    
    kobo_info = {}
    
    # タイトルを追加
    kobo_title = soup_li[n].find_all('h3')
    kobo_title = kobo_title[0].text.replace('\u3000', '')
    kobo_info.setdefault('タイトル', kobo_title)
    
    # 応募リンクを追加
    obo_link = ''
    # 川柳の詳細ページのHTML要素の<a>タグのclass='btn-koushiki'の要素を取得
    obo_link = soup_kobo.find_all('a', attrs={'class': 'btn-koushiki'})
    obo_link = obo_link[0].get('href')
    # 応募リンクをkobo_infoに追加
    kobo_info.setdefault('応募リンク', obo_link)
    
    # 公募概要情報の取得・配列に格納
    for j in range(len(soup_kobo_dd)):
        kobo_feature_title = soup_kobo_dt[j].text
        kobo_feature_title = kobo_feature_title.replace('\u3000', ' ')
        
        # 項目名をデータベースのカラム名に変換
        if '締切' == kobo_feature_title:
            kobo_feature_title = '締切日'
        elif '賞' == kobo_feature_title:
            kobo_feature_title = '賞金'
        elif '募集内容' == kobo_feature_title:
            kobo_feature_title = '募集内容'
        elif '参加資格' == kobo_feature_title:
            kobo_feature_title = '応募資格'
        elif '主催' == kobo_feature_title:
            kobo_feature_title = '主催'
        else:
            continue
        
        # HTML要素のテキストだけを取り出す
        kobo_feature = soup_kobo_dd[j].text.replace('\u3000', ' ')
        kobo_feature = kobo_feature.replace('\n', '')
        kobo_feature = kobo_feature.replace('\t', '')
        kobo_feature = kobo_feature.replace('\r', '')
        kobo_info.setdefault(kobo_feature_title, kobo_feature)
        
    # kobo_infoの中身を並び替える
    kobo_sort = {}
    kobo_sort.setdefault('タイトル', kobo_info['タイトル'])
    kobo_sort.setdefault('締切日', kobo_info['締切日'])
    kobo_sort.setdefault('賞金', kobo_info['賞金'])
    kobo_sort.setdefault('募集内容', kobo_info['募集内容'])
    kobo_sort.setdefault('応募資格', kobo_info['応募資格'])
    kobo_sort.setdefault('主催', kobo_info['主催'])
    kobo_sort.setdefault('応募リンク', kobo_info['応募リンク'])
    
    # 公募情報を配列に格納
    kobo_info_list.append(kobo_sort)
    
    # サーバへの負荷を下げるためにsleep
    time.sleep(3)
    
    
# csvで出力
import pandas as pd

df = pd.DataFrame(kobo_info_list)

# excelで文字化けしないようにutf_8_sigでエンコード
df.to_csv('kobo_scraping.csv', encoding='utf_8_sig', index = False)

In [39]:
# 確認用
# kobo_sort

In [38]:
# 確認用
# kobo_info_list

In [2]:
# 確認用
df_read = pd.read_csv('kobo_scraping.csv', index_col=0)
df_read

,締切日,賞金,募集内容,応募資格,主催,応募リンク
タイトル,,,,,,
第6回 オリックス 働くパパママ川柳 作品募集,2022年02月11日 (金)作品提出・応募締切、必着,●大賞（1句） 賞金20万円、オリックスグループが運営する4施設から選べる1泊2食付きホテル...,働きながら子育てに奮闘するパパとママの日常をテーマとする川柳※応募者の自作未発表作品に限る,既婚未婚、年齢不問,オリックスグループ,https://www.orix.co.jp/grp/move_on/entry/orix-...
第3回 私のSDGsコンテスト,2022年02月13日 (日)作品提出・応募締切、必着,●大賞 10万円分の図書カード●シャボン玉石けん賞 賞品●北九州市長賞 賞品●朝日新聞社賞 ...,テーマにそった川柳または写真※未発表のオリジナル作品に限る【テーマ】17あるSDGsの目標（...,不問,シャボン玉石けん株式会社、株式会社朝日新聞社、北九州市（SDGs未来都市）,https://www.asahi.com/ads/my-sdgs-contest/
第4回 おウチde俳句大賞,2022年02月14日 (月)作品提出・応募締切、必着,●大賞（1名） 賞金20万円、記念盾●各テーマ賞（6名） 賞金5万円、記念盾※大賞は各テーマ...,テーマにそった自作未発表の俳句※春・夏・秋・冬・新年、いつの季語を使用しても可【テーマ】（1...,不問,株式会社朝日出版社,https://www.asahipress.com/special/ouchidehaik...
第21回 雪のラブレター募集,2022年02月14日 (月)作品提出・応募締切、消印有効,●大賞（各部門1作品） 賞状、3万円相当の尾花沢特産品詰合せ●優秀賞（各部門2作品） 賞状、...,雪にまつわる恋の俳句、または恋文※自作・未発表のみとし、偽名応募・類似応募は不可【部門】（1...,不問,山形県尾花沢市共催：一般社団法人 尾花沢市沢市観光物産協会,http://www.city.obanazawa.yamagata.jp/15398.html
親と子のホンネ俳句大賞,2022年02月15日 (火)作品提出・応募締切、必着,●最優秀賞（1名） 10万円分のQUOカード●優秀賞（2名） 5万円分のQUOカード●佳作（...,テーマにそった未発表の俳句※季語や字余り等にとらわれずに自由に表現すること※応募者本人が一切...,日本国内在住の方に限る※応募の前に応募規約を必ず確認し、同意のうえ参加すること※未成年の方は...,東日本旅客鉄道株式会社、東京都交通局、東京地下鉄株式会社、セントラル警備保障株式会社,https://www.mamorail.jp/haiku/
第2回 めざせきのこ博士！げんきのコンクール《小学生以下限定》,2022年02月21日 (月)作品提出・応募締切、必着,●最優秀賞（各1点） 3万円分の図書カード●優秀賞（各1点） 5000円分の図書カード※参加...,きのこ体操の動画・写真、またはきのこ学習の感想文・川柳【きのこ体操】※きのこ体操の体操風景を...,【きのこ体操】全国の未就学児、小学生※クラスやスクール等の団体でも個人でも可【きのこ学習】全...,ホクト株式会社,https://kinkatsu-kinoko.nichibunkyo.jp
第35回 うなぎ川柳大会,2022年02月25日 (金)作品提出・応募締切、必着,●金賞（1点） 3万円分の図書カード他賞あり,うなぎにまつわる未発表の川柳,不問,うなぎ百撰,https://www.unagi.co.jp/第28回うなぎ川柳大会%E3%80%80作品...
日本写真俳句大賞 2022,2022年02月28日 (月)作品提出・応募締切、必着,●最優秀賞（1名） 3万円分の図書券●優秀賞（2名） 3000円分の図書券●入選（12名）※...,自作未発表の写真俳句※写真と俳句、川柳等、五七五の十七音から成る日本語の定型詩を組み合わせた...,不問,写真俳句連絡協議会,https://shashin-haiku.org/contest/
高血圧に関する川柳と標語コンテスト 2022,2022年02月28日 (月)作品提出・応募締切、必着,【川柳部門・標語部門】●最優秀賞（川柳・標語それぞれ1作品） 賞金10万円、副賞、表彰盾●日...,テーマにそった川柳や標語※本人が創作した未発表作品に限る※応募後、賞の発表まで他に公表しない...,不問※チーム部門は、1チーム2～5名とし、うち1名以上は応募時点で満13～23歳とする※未成...,日本高血圧学会、日本高血圧協会,https://www.jpnsh.jp/info_senryu.html


In [ ]:
import requests
import KOBO_URL
import time
from bs4 import BeautifulSoup

# スクレイピング対象のサイトのURLを取得
url1 = KOBO_URL.KOBO_URL1
res = requests.get(url1)

In [2]:
# WebサイトのHTML要素を取得
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
# soup

In [4]:
# li要素を取得
soup_li = soup.find_all('li', attrs={'class': 'contest-list-item'})

In [6]:
# h3要素を取得
# soup_h3 = soup_li[0].find_all('h3')

In [7]:
# soup_h3

In [8]:
# a要素を取得
# soup_a = soup_li[0].find('a')
# soup_a = soup_a.get('href')

In [9]:
# soup_a

In [10]:
# 応募リンクを配列に格納する
link_list = []
while_cnt = 0
while True:
    try:
        soup_a = soup_li[while_cnt].find('a')
        soup_a = soup_a.get('href')
        link_list.append(soup_a)
        
    except IndexError as e:
        break
        
    while_cnt += 1

In [11]:
# link_list

In [12]:
# soup_a_cnt

In [13]:
kobo_url = link_list[0]

In [14]:
# kobo_url

In [15]:
res_kobo = requests.get(kobo_url)

In [16]:
# res_kobo

In [17]:
# 公募の詳細ページのHTML要素を取得
soup_kobo = BeautifulSoup(res_kobo.text, 'html.parser')

In [18]:
# soup_kobo

In [19]:
soup_kobo_dl = soup_kobo.find_all('dl')

In [20]:
# soup_kobo_dl

In [21]:
# n = 0
# for n in range(30):
#     soup_kobo_dt = soup_kobo_dl[n].find_all('dt')
        
#     if '締切' == soup_kobo_dt[0].text:
#         break
        
# soup_kobo_dd = soup_kobo_dl[n].find_all('dd')
# soup_kobo_dd

In [22]:
while_cnt = 0
while True:
    try:
        soup_kobo_dt = soup_kobo_dl[while_cnt].find_all('dt')
        
        if '締切' == soup_kobo_dt[0].text:
            break
        
    except IndexError as e:
        break
        
    while_cnt += 1

In [23]:
soup_kobo_dt = soup_kobo_dl[while_cnt].find_all('dt')
soup_kobo_dd = soup_kobo_dl[while_cnt].find_all('dd')

In [24]:
# soup_kobo_dt[0].text

In [25]:
# soup_kobo_dd

In [26]:
# kobo_title = soup_kobo_dt[0].text
# kobo_feature = soup_kobo_dd[0].text.replace('\n', '')
# kobo_feature = kobo_feature.replace('\t', '')

kobo_title_list = []
kobo_feature_list = []

In [27]:
# len(soup_kobo_dd)

In [28]:
for n in range(len(soup_kobo_dd)):
    kobo_title = soup_kobo_dt[n].text
    kobo_feature = soup_kobo_dd[n].text.replace('\n', '')
    kobo_feature = kobo_feature.replace('\t', '')
    kobo_title_list.append(kobo_title)
    kobo_feature_list.append(kobo_feature)
    

In [29]:
# kobo_title_list

In [30]:
# kobo_feature_list

In [31]:
# kobo_info = {}

# for n in range(len(soup_kobo_dd)):
#     kobo_feature_title = soup_kobo_dt[n].text
#     kobo_feature = soup_kobo_dd[n].text.replace('\n', '')
#     kobo_feature = kobo_feature.replace('\t', '')
#     kobo_feature = kobo_feature.replace('\u3000', ' ')
#     # kobo_info.setdefault(soup_kobo_dt[n].text, kobo_feature)
#     kobo_info.setdefault(kobo_feature_title, kobo_feature)

In [32]:
# soup_kobo_dt

In [33]:
# soup_kobo_dt[0].text

In [34]:
# soup_kobo_dd[1].text

In [35]:
# kobo_info_dict = {}
kobo_info_list = []

for n in range(len(link_list)):
# for n in range(2):
    # 情報を取得する公募ページに移動してHTTPを取得
    kobo_url = link_list[n]
    res_kobo = requests.get(kobo_url)
    soup_kobo = BeautifulSoup(res_kobo.text, 'html.parser')
    soup_kobo_dl = soup_kobo.find_all('dl')
    # soup_kobo_dt = soup_kobo_dl[3].find_all('dt')
    # soup_kobo_dd = soup_kobo_dl[3].find_all('dd')
    
    while_cnt = 0
    while True:
        try:
            soup_kobo_dt = soup_kobo_dl[while_cnt].find_all('dt')
            if '締切' == soup_kobo_dt[0].text:
                break
        except IndexError as e:
            break
        while_cnt += 1
    
    soup_kobo_dd = soup_kobo_dl[while_cnt].find_all('dd')
    
    kobo_info = {}
    
    # タイトルを追加
    kobo_title = soup_li[n].find_all('h3')
    kobo_title = kobo_title[0].text.replace('\u3000', '')
    kobo_info.setdefault('タイトル', kobo_title)
    
    # 応募リンクを追加
    obo_link = ''
    # 川柳の詳細ページのHTML要素の<a>タグのclass='btn-koushiki'の要素を取得
    obo_link = soup_kobo.find_all('a', attrs={'class': 'btn-koushiki'})
    obo_link = obo_link[0].get('href')
    # 応募リンクをkobo_infoに追加
    kobo_info.setdefault('応募リンク', obo_link)
    
    # 公募概要情報の取得・配列に格納
    for j in range(len(soup_kobo_dd)):
        kobo_feature_title = soup_kobo_dt[j].text
        kobo_feature_title = kobo_feature_title.replace('\u3000', ' ')
        
        # 項目名をデータベースのカラム名に変換
        if '締切' == kobo_feature_title:
            kobo_feature_title = '締切日'
        elif '賞' == kobo_feature_title:
            kobo_feature_title = '賞金'
        elif '募集内容' == kobo_feature_title:
            kobo_feature_title = '募集内容'
        elif '参加資格' == kobo_feature_title:
            kobo_feature_title = '応募資格'
        elif '主催' == kobo_feature_title:
            kobo_feature_title = '主催'
        else:
            continue
        
        # HTML要素のテキストだけを取り出す
        kobo_feature = soup_kobo_dd[j].text.replace('\u3000', ' ')
        kobo_feature = kobo_feature.replace('\n', '')
        kobo_feature = kobo_feature.replace('\t', '')
        kobo_feature = kobo_feature.replace('\r', '')
        kobo_info.setdefault(kobo_feature_title, kobo_feature)
        
    # kobo_infoの中身を並び替える
    kobo_sort = {}
    kobo_sort.setdefault('タイトル', kobo_info['タイトル'])
    kobo_sort.setdefault('締切日', kobo_info['締切日'])
    kobo_sort.setdefault('賞金', kobo_info['賞金'])
    kobo_sort.setdefault('募集内容', kobo_info['募集内容'])
    kobo_sort.setdefault('応募資格', kobo_info['応募資格'])
    kobo_sort.setdefault('主催', kobo_info['主催'])
    kobo_sort.setdefault('応募リンク', kobo_info['応募リンク'])
    
    # 公募情報を配列に格納
    # kobo_title = soup_li[n].find_all('h3')
    # kobo_title = kobo_title[0].text.replace('\u3000', '')
    # kobo_info_dict.setdefault(kobo_title, kobo_info)
    
    kobo_info_list.append(kobo_sort)
    
    # サーバへの負荷を下げるためにsleep
    time.sleep(3)

In [36]:
# 確認用
# kobo_sort

In [37]:
# 確認用
# kobo_info_list

In [33]:
# csvで出力
import pandas as pd

df = pd.DataFrame(kobo_info_list)

# excelで文字化けしないようにutf_8_sigでエンコード
df.to_csv('kobo_scraping.csv', encoding='utf_8_sig', index = False)

In [36]:
# kobo_info_list

In [37]:
# df_read = pd.read_csv('kobo_scraping.csv', index_col=0)
# df_read

In [84]:
# 締切日をchar型からdate型に変換する

str_date = '2022年02月11日 (金)作品提出・応募締切、必着'
str_date

'2022年02月11日 (金)作品提出・応募締切、必着'

In [85]:
import re

# pattern = '(20\d\d.月)'
pattern = '20\d\d.\d\d.\d\d.'

str_date = re.match(pattern, str_date)
str_date =  str(str_date.group())
str_date

'2022年02月11日'

In [86]:
str_date = str_date.replace('年', '-')
str_date = str_date.replace('月', '-')
str_date = str_date.replace('日', '')
str_date

'2022-02-11'

In [78]:
from datetime import datetime

mydate = datetime.strptime(str_date, '%Y年%m月%d日')
print(mydate)

2022-02-11 00:00:00
